In [ ]:
%matplotlib nbagg

In [ ]:
import os
import pandas as pd
import datetime
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from patsy import dmatrices
import matplotlib.pyplot as plt

In [ ]:
DATADIR = os.path.join(os.path.expanduser("~"), "DATA", "TimeSeries", "EPA")
os.path.exists(DATADIR)

In [ ]:
files = os.listdir(DATADIR)
files

## Read the air quality data

In [ ]:
slc = pd.read_excel(os.path.join(DATADIR, 'Salt_Lake_2016_PM25.xlsx'))
print(slc.columns)
slc.head()

In [ ]:
d1 = slc["Date Local"][0]
t1 = slc["Time Local"][0]
print("Date Local type",type(d1),"Time Local type",type(t1))
datetime.datetime.combine(d1.to_datetime(), t1)


### Comments:

* Dates and times are split into separate columns
* We have both local time and UTC time 

## Merging Dates and Time
* After we have read in the data we could combine the dates and times
    * Convert pandas Timestamp to datetime
    * Use ``datetime.combine`` to merge with ``datetime.time``
* We can also merge these when reading the data by specifying the ``parse_dates`` keyword

#### Using ``datetime.combine``

In [ ]:
slc.apply(lambda x: datetime.datetime.combine(x["Date Local"],x["Time Local"]), axis=1).head()

In [ ]:
sd = pd.read_excel(os.path.join(DATADIR, "San_Diego_2016_PM25.xlsx"))
sd.head()

## Read in weather data

In [ ]:
slc_weather = pd.read_excel(os.path.join(DATADIR, 'SLC_Weather_2016.xlsx'), skiprows=[1])
slc_weather.head()

In [ ]:
slc_weather.columns

In [ ]:
slc_weather['Day'][0]

## Our Weather has Resolution of Days
## Our pollutant data has resolution of hours
## Group and take mean?

In [ ]:
slc.groupby("Date Local", as_index=False).aggregate(np.mean)

### Group and take sum?

In [ ]:
slc.groupby("Date Local", as_index=False).aggregate(np.sum)

### Applying different functions to different columns

In [ ]:
slc_day = slc.groupby(["Date Local", "Site Num"], 
                      as_index=False).aggregate(
                         {'Sample Measurement' : np.mean,
                          'MDL': np.median})
slc_day.head()

In [ ]:
slc_day_all = slc_day.merge(slc_weather, 
                            left_on="Date Local", 
                            right_on="Day")
slc_day_all.head()

### Rename columns to eliminate spaces

In [ ]:
{c:c.replace(" ", "_") for c in slc_day_all.columns}

In [ ]:
slc_final = slc_day_all.rename(
    columns={c:c.replace(" ", "_") for c in slc_day_all.columns}
)

### What would be an interesting plot?